# YOLOv11 Segmentation Training on Kaggle P100 GPU
## Surgical Organ Detection & Segmentation

This notebook trains a YOLOv11n-seg model on Kaggle's P100 GPU (16GB VRAM) for detecting and segmenting anatomical structures during surgical operations.

**Dataset Classes:**
- External Iliac Artery
- External Iliac Vein
- Obturator Nerve
- Ovary
- Ureter
- Uterine Artery
- Uterus

**Training Configuration:**
- Model: YOLOv11n-seg (nano)
- Batch Size: 16 (optimized for P100)
- Image Size: 640x640
- Epochs: 150
- Expected Training Time: ~2-3 hours

## 1. Check GPU Availability

In [ ]:
!nvidia-smi

import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ No GPU detected! Make sure GPU accelerator is enabled in Kaggle settings.")

## 2. Install Ultralytics YOLOv11

In [ ]:
!pip install -q ultralytics
print("✅ Ultralytics installed successfully!")

## 3. Import Libraries

In [ ]:
import os
import torch
from ultralytics import YOLO
from pathlib import Path

# Set random seed for reproducibility
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Configure memory allocation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

print("✅ Libraries imported successfully!")

## 4. Upload and Verify Dataset

**Instructions for Kaggle:**
1. Upload your `medhack_yolov11` dataset as a Kaggle Dataset
2. Add it to this notebook in "Input" section
3. The dataset should be available at `/kaggle/input/your-dataset-name/`

Or, modify the path below to match your dataset location.

In [ ]:
# Check Kaggle input directory
print("Available datasets in /kaggle/input:")
if os.path.exists('/kaggle/input'):
    for item in os.listdir('/kaggle/input'):
        print(f"  - {item}")
else:
    print("  ⚠️ Not running on Kaggle - using local path")

# Set dataset path - MODIFY THIS to match your dataset location
DATASET_PATH = '/kaggle/input/medhack-yolov11'  # Change to your dataset name
DATA_YAML = f'{DATASET_PATH}/data.yaml'

# Verify dataset structure
if os.path.exists(DATASET_PATH):
    print(f"\n✅ Dataset found at: {DATASET_PATH}")
    print("\nDataset structure:")
    for root, dirs, files in os.walk(DATASET_PATH):
        level = root.replace(DATASET_PATH, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f'{indent}{os.path.basename(root)}/')
        if level < 2:  # Only show 2 levels deep
            sub_indent = ' ' * 2 * (level + 1)
            for file in files[:3]:  # Show first 3 files
                print(f'{sub_indent}{file}')
            if len(files) > 3:
                print(f'{sub_indent}... and {len(files)-3} more files')
else:
    print(f"\n❌ Dataset not found at: {DATASET_PATH}")
    print("Please update DATASET_PATH variable to match your dataset location")

## 5. Load YOLOv11n-seg Model

In [ ]:
# Load pretrained YOLOv11n-seg model
print("Loading YOLOv11n-seg model with pretrained weights...")
model = YOLO('yolo11n-seg.pt')
print("✅ Model loaded successfully!")

## 6. Configure Training Parameters (P100 Optimized)

In [ ]:
# Training configuration optimized for P100 (16GB VRAM)
training_config = {
    # Dataset
    'data': DATA_YAML,
    
    # Training parameters
    'epochs': 150,
    'batch': 16,  # P100 can handle larger batches
    'imgsz': 640,
    'device': 0,  # GPU
    'project': '/kaggle/working/runs/segment',
    'name': 'surgical_organs_p100',
    'exist_ok': True,
    'verbose': False,
    
    # Optimization
    'optimizer': 'AdamW',
    'lr0': 0.001,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'cos_lr': True,
    
    # Performance
    'amp': True,  # Mixed precision
    'workers': 8,
    'cache': False,
    'multi_scale': True,
    
    # Medical-specific augmentation (conservative)
    'hsv_h': 0.010,
    'hsv_s': 0.5,
    'hsv_v': 0.3,
    'degrees': 5.0,
    'translate': 0.1,
    'scale': 0.3,
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,
    'fliplr': 0.5,
    'mosaic': 0.8,
    'mixup': 0.1,
    'copy_paste': 0.2,
    'auto_augment': 'randaugment',
    'erasing': 0.2,
    
    # Validation & Saving
    'val': True,
    'save': True,
    'save_period': 10,
    'patience': 30,
    'plots': True,
    
    # Segmentation
    'overlap_mask': True,
    'mask_ratio': 4,
    
    # Loss weights
    'box': 7.5,
    'cls': 0.5,
    'dfl': 1.5,
    
    # Other
    'close_mosaic': 10,
    'seed': 42,
}

print("Training Configuration:")
print(f"  Model: YOLOv11n-seg")
print(f"  Epochs: {training_config['epochs']}")
print(f"  Batch Size: {training_config['batch']}")
print(f"  Image Size: {training_config['imgsz']}")
print(f"  Device: GPU (P100)")
print(f"  Expected Time: 2-3 hours")

## 7. Train Model

This will take approximately 2-3 hours on P100 GPU.

In [ ]:
# Start training
print("="*60)
print("Starting Training...")
print("="*60)

results = model.train(**training_config)

print("\n" + "="*60)
print("✅ Training Complete!")
print("="*60)

## 8. Validate Trained Model

In [ ]:
# Load best model and validate
best_model_path = '/kaggle/working/runs/segment/surgical_organs_p100/weights/best.pt'
best_model = YOLO(best_model_path)

print("Validating best model...")
metrics = best_model.val()

print("\n" + "="*60)
print("Validation Metrics:")
print("="*60)
print(f"Box Detection:")
print(f"  mAP50:     {metrics.box.map50:.4f}")
print(f"  mAP50-95:  {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall:    {metrics.box.mr:.4f}")
print(f"\nSegmentation:")
print(f"  mAP50:     {metrics.seg.map50:.4f}")
print(f"  mAP50-95:  {metrics.seg.map:.4f}")
print(f"  Precision: {metrics.seg.mp:.4f}")
print(f"  Recall:    {metrics.seg.mr:.4f}")
print("="*60)

## 9. Display Training Results

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Image, display

# Display training results plot
results_path = '/kaggle/working/runs/segment/surgical_organs_p100/results.png'
if os.path.exists(results_path):
    print("Training Curves:")
    display(Image(filename=results_path))
else:
    print("⚠️ Results plot not found")

# Display confusion matrix
confusion_path = '/kaggle/working/runs/segment/surgical_organs_p100/confusion_matrix.png'
if os.path.exists(confusion_path):
    print("\nConfusion Matrix:")
    display(Image(filename=confusion_path))

# Display validation batch predictions
val_batch_path = '/kaggle/working/runs/segment/surgical_organs_p100/val_batch0_pred.jpg'
if os.path.exists(val_batch_path):
    print("\nValidation Predictions:")
    display(Image(filename=val_batch_path))

## 10. Save Model Weights

The trained weights are automatically saved and can be downloaded from Kaggle's output section.

In [ ]:
# List saved model files
weights_dir = '/kaggle/working/runs/segment/surgical_organs_p100/weights'
print("Saved Model Weights:")
print("="*60)

if os.path.exists(weights_dir):
    for file in os.listdir(weights_dir):
        file_path = os.path.join(weights_dir, file)
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"  {file:20s} - {size_mb:.2f} MB")
    
    print("\n✅ Download these files from Kaggle Output:")
    print(f"   best.pt  - Best model based on validation metrics")
    print(f"   last.pt  - Final epoch checkpoint")
else:
    print("❌ Weights directory not found")

print("="*60)
print("\n📥 To download: Click 'Output' tab → Download files")
print("   Or use Kaggle API to download programmatically")

## 11. Export Model to ONNX (Optional)

Export to ONNX format for faster CPU inference in production.

In [ ]:
# Export to ONNX format
print("Exporting model to ONNX format...")
try:
    onnx_path = best_model.export(format='onnx', imgsz=640)
    print(f"✅ ONNX model exported successfully!")
    print(f"   Location: {onnx_path}")
    print(f"   This format provides 20-30% faster CPU inference")
except Exception as e:
    print(f"❌ Export failed: {e}")

## 12. Test Inference on Sample Image (Optional)

In [ ]:
# Test inference on a sample test image
test_image_dir = f'{DATASET_PATH}/test/images'

if os.path.exists(test_image_dir):
    # Get first test image
    test_images = [f for f in os.listdir(test_image_dir) if f.endswith(('.jpg', '.png'))]
    if test_images:
        test_img_path = os.path.join(test_image_dir, test_images[0])
        
        print(f"Running inference on: {test_images[0]}")
        results = best_model.predict(test_img_path, conf=0.25, save=True, project='/kaggle/working')
        
        # Display result
        result_path = '/kaggle/working/predict/image0.jpg'
        if os.path.exists(result_path):
            print("\nInference Result:")
            display(Image(filename=result_path))
        
        print(f"\n✅ Detected {len(results[0].boxes)} objects")
    else:
        print("No test images found")
else:
    print(f"Test directory not found: {test_image_dir}")

## Summary

✅ **Training Complete!**

**Model Files Available in Output:**
- `best.pt` - Best model weights (download this!)
- `last.pt` - Last epoch checkpoint
- `best.onnx` - ONNX format (if exported)

**Next Steps:**
1. Download `best.pt` from Kaggle Output
2. Use the model locally:
   ```python
   from ultralytics import YOLO
   model = YOLO('best.pt')
   results = model.predict('image.jpg')
   ```

**Expected Performance:**
- Training Time: ~2-3 hours on P100
- mAP50 (Seg): 0.65-0.80
- Inference Speed: 15-25 FPS on GPU, 3-5 FPS on CPU

**Hardware Used:**
- GPU: NVIDIA P100 (16GB)
- Batch Size: 16
- Optimized for medical organ segmentation